In [3]:
from torchrl.data.replay_buffers import (
    TensorDictReplayBuffer,
    LazyTensorStorage,
    PrioritizedSampler,
)
from tensordict import TensorDict

import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import gymnasium as gym
import numpy as np

from rlarcworld.arc_dataset import ArcDataset, ArcSampleTransformer
from rlarcworld.enviroments.arc_batch_grid_env import ArcBatchGridEnv
from rlarcworld.enviroments.wrappers.rewards import PixelAwareRewardWrapper
from rlarcworld.agent.actor import ArcActorNetwork
from rlarcworld.agent.critic import ArcCriticNetwork

from rlarcworld.algorithms.d4pg import D4PG

2025-02-14 12:27:29 - arc_batch_grid_env.py - INFO - Registering gymnasium environment


In [5]:
env = ArcBatchGridEnv(
    size=30,
    color_values=11,
    n_steps=1,
    gamma=1.0,
)

env = PixelAwareRewardWrapper(env, n_steps=1, gamma=1.0)
isinstance(env, PixelAwareRewardWrapper), isinstance(env.env, gym.Env)

(True, True)

In [8]:
dataset = ArcDataset(
            arc_dataset_dir="./dataset/training",
            keep_in_memory=True,
            transform=ArcSampleTransformer(
                (30, 30), examples_stack_dim=10
            ),
        )
train_samples = DataLoader(dataset=dataset, batch_size=10)

In [9]:
len(dataset)

416

In [4]:
for train_sample in train_samples:
    pass    

In [ ]:
def sampler(max_s):
    sample = 0
    x = {"flag": False}   
    while sample < max_s:
        x["i"] = sample
        yield x
        sample += 1
gener = sampler(10)
for s in gener:
    print(s)
    if s["i"] == 5:
        break
config["truncate"] = True
for s in gener:
    print(s)

{'truncate': False}
0
{'truncate': False}
1
{'truncate': False}
2
{'truncate': False}
3
{'truncate': False}
4
{'truncate': False}
5
{'truncate': True}
6
{'truncate': True}
7
{'truncate': True}
8
{'truncate': True}
9


In [3]:
flag = False
gener = sampler(10, truncate=flag)
print(next(gener))
print(next(gener))
flag = True
print(next(gener))
print(next(gener))

False
0
False
1
False
2
False
3
